In [1]:
import geopandas as gpd
from osgeo import ogr
import pandas as pd
import tifffile as tiff #needed for the tif data for perry county
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from math import sin, cos, sqrt, atan2, radians
import sys
from shapely.geometry import Polygon, box, Point, LineString, MultiLineString
import pickle
import gurobipy as gp
from gurobipy import GRB
import tifffile as tiff #needed for the tif data for perry county
import xlwt
from xlwt import Workbook
from shapely.ops import snap, split, nearest_points
#from shapely.geometry import MultiPoint, LineString
#from dbfread import DBF
import osmnx as ox
import networkx as nx
import math
import gurobipy as gp
from gurobipy import GRB
import os
import csv
from sklearn.cluster import AgglomerativeClustering
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

In [2]:
# calculate distance
def haversinedist(lat1, lon1, lat2, lon2):
    R = 6373.0
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance #unit in km

In [3]:
# read data
def readClusterFile(fileID):
    file = np.genfromtxt(fileID, delimiter=",", skip_header = 1)
    file = file[:,1:]
    return file
clusterfile = 'Centralized_elevcluster' + str(1) + '.csv'

In [4]:
os.chdir("/Users/yuelanzhu/Downloads/Research/my code file")
# read data
def readClusterFile(fileID):
    file = np.genfromtxt(fileID, delimiter=",", skip_header = 1)
    file = file[:,1:]
    return file
clusterfile = 'Centralized_elevcluster' + str(1) + '.csv'
building_coords = readClusterFile(clusterfile)

In [5]:
# pipe parameters
pipesize = [0.05, 0.06, 0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6] #m
pipesize_str, pipecost = gp.multidict({'0.05': 8.7, '0.06': 9.5, '0.08': 11, \
                                                   '0.1': 12.6, '0.15': 43.5,'0.2': 141, '0.25': 151, '0.3': 161, '0.35':230, '0.4': 246, '0.45':262, 
                                                   '0.5':292, '0.6':315}) #$/m
excavation = 25
bedding_cost_sq_ft = 6
capital_cost_pump_station = 166000
ps_flow_cost = 0.19
ps_OM_cost = 175950

In [6]:
# creat MST

graph = []
mstree = []
def createMSTnx(dataframe,n):
    #only takes longitude and latitude from the cluster data, note we also have elevation and stuff there too
    #coordinates = twoDcluster
    cluster = AgglomerativeClustering(n_clusters=n, affinity='euclidean', linkage='ward')
    cluster.fit_predict(building_coords[:,1:2])
    # add cluster to dataframe
    dataframe['cluster.labels_'] = cluster.labels_.tolist()
    for k in range(n): 
        cluster_label = dataframe[dataframe['cluster.labels_'] == k]
        cluster_label = cluster_label[['longitude','latitude']]
        latlon = cluster_label.to_numpy()
        nrows, ncols = latlon.shape
        #creates graph
        graph.append(nx.Graph())
        weights = []
        #distance in km as weight between each point in the graph
        for i in range(nrows):
            graph[k].add_node(i,pos=(latlon[i,0],latlon[i,1]))
            for j in range(i+1,nrows):
                dist = haversinedist(latlon[i,1], latlon[i,0], latlon[j,1], latlon[j,0])
                weights.append(dist)
                graph[k].add_edge(i,j, weight = dist)
        #creates MST
        mstree.append(nx.minimum_spanning_tree(graph[k]))
    return mstree, graph

In [7]:
def result(df, num_c, cluster_labels, pipesize, pipecost, water_demand, HLmax, excavation, bedding_cost_sq_ft, capital_cost_pump_station, ps_OM_cost):

    # select highest and lowest nodes in cluster
    tank = df.loc[df.groupby(['cluster.labels_'])['elevation'].idxmax()].reset_index(drop=True)
    trem = df.loc[df.groupby(['cluster.labels_'])['elevation'].idxmin()].reset_index(drop=True)
                      
     # find label of tank node in cluster
    tank_lon = str(tank[tank['cluster.labels_'] == cluster_labels][['longitude']].iat[0,0])
    tank_lat = str(tank[tank['cluster.labels_'] == cluster_labels][['latitude']].iat[0,0])
    global tank_node
    for i in mstree[cluster_labels].nodes():
        if str(mstree[cluster_labels].nodes[i]['pos'][0]) == tank_lon and str(mstree[cluster_labels].nodes[i]['pos'][1]) == tank_lat:
            tank_node = i                  
    #create flow direction dict
    links = list(mstree[cluster_labels].edges)
    flow_dir = {}
    for i,j in links:
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) < len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node i is nearer to tank
           flow_dir[i,j]=(i,j)
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
         # if node j is nearer to tank
           flow_dir[i,j]=(j,i) 
    # find nodes for inflow and outflow
    innode = {}
    outnode = {}
    for i,j in links:
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
           # if node j is nearer to tank
            innode.setdefault(i, [])
            innode[i].append(j)
            outnode.setdefault(j, [])
            outnode[j].append(i)
        else:
           # if node i is nearer to tank
            innode.setdefault(j, [])
            innode[j].append(i)
            outnode.setdefault(i, [])
            outnode[i].append(j)
    for i in mstree[cluster_labels].nodes():
        innode.setdefault(i,[])
        outnode.setdefault(i, [])
   # directed graph
    mstree_dir = nx.DiGraph()
    mstree_dir.add_nodes_from(mstree[cluster_labels])
    mstree_dir.add_edges_from(flow_dir.values())
    weights = {}
    for i in list(mstree_dir.edges()):
        weights[i] = None
    for i,j in flow_dir:
        if flow_dir[i,j]==(i,j):
            weights[i,j]=mstree[cluster_labels].edges[i,j]['weight']
        if flow_dir[i,j]==(j,i):
            weights[j,i]=mstree[cluster_labels].edges[i,j]['weight']
    nx.set_edge_attributes(mstree_dir, values = 1, name = 'weight')
    nx.set_edge_attributes(mstree_dir, values = weights, name = 'weight')
    # determine iterate order
    walk = {}
    for i,j in mstree_dir.edges():
        walk[i,j] = len(nx.shortest_path(mstree_dir ,source=tank_node, target=j))
    sorted_walk = dict(sorted(walk.items(), key=lambda item: item[1], reverse = True))
    # pipe flow
    pipe_flow = {}
    for i,j in sorted_walk:
        if len(outnode[j]) == 0:
            pipe_flow[i,j] = 1.095E-5
        else:
            pipe_flow[i,j] = sum(pipe_flow[m,n] for m,n in sorted_walk if m == j) + 1.095E-5
    # head loss at largest dia        
    head_loss = {}
    for i in list(mstree_dir.edges()):
        head_loss[i] = None
    for i,j in list(mstree_dir.edges()):
        head_loss[i,j] = 10700*(pipe_flow[i,j]/140)**1.852*0.6*1000*mstree_dir[i][j]["weight"]
    # flow, distance and head loss for each edge
    edge_list = {}
    for i,j in links:
        if (i,j) == flow_dir[i,j]:
            edge_list[i,j] =[pipe_flow[i,j],mstree[cluster_labels][i][j]["weight"], head_loss[i,j]]
        else:
            edge_list[j,i] =[pipe_flow[j,i],mstree[cluster_labels][i][j]["weight"], head_loss[j,i]]
   #node elevation excavation in meters
   #upper bound is arbritrary maximum depth assuming 1 foot or 0.3048 meters of cover beneath the surface is needed for the pipes
   #a lower bound variable is created but not used. In future models might need to implement that depending on the site (digging too deep for excavation is not feasible for many projects)
    cluster = df[df['cluster.labels_'] == cluster_labels]
    cluster_elv = cluster[['elevation']]
    elevation_ub = dict()
    elevation_lb = dict()
    for i in range(len(cluster_elv)):
        GE=float(str(cluster_elv.values.tolist()[i])[1:-1])
        elevation_ub[i] = GE - 0.3
        elevation_lb[i] = GE - 30 #10
    GE=[]
    for i in range(len(cluster_elv)):
        GE.append(float(str(cluster_elv.values.tolist()[i])[1:-1]))
    #for i in range(mstree[cluster_labels].number_of_nodes()):
        #elevation_ub[i] = cluster_elv.iloc[[0]] - 0.3048
        #elevation_lb[i] = cluster_elv.iloc[[0]] - 30
        
    m = gp.Model('pipe and pump distribution')
    m.Params.timeLimit = 12000
   # binary variable indicating if at link ij pipe of diameter k is implemented
    d = m.addVars(mstree_dir.edges(), pipesize, vtype = GRB.BINARY, name = "diameter")
   #continuous variable representing the hydro loss in link ij.
    HL = m.addVars(mstree_dir.edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "hydro loss")
    # hydro gradient
    #J = m.addVars(mstree[cluster_labels].edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "hydro gradient")
   # binary variable indicating if at link ij the pump of type m is implemented.
    p = m.addVars(mstree_dir.edges(), vtype = GRB.BINARY, name = "PUMP")
   #continuous variable representing the pressure injected by the pump at link ij.
    pr = m.addVars(mstree_dir.edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "PRESSURE BY PUMP")
   #pipe elevations at node i 
    e = m.addVars(mstree_dir.nodes(), lb = 0, vtype = GRB.CONTINUOUS, name = 'In Node Elevation')
   #continuous variable representing the head pressure at node i
    H = m.addVars(mstree_dir.nodes(),lb = 0, vtype = GRB.CONTINUOUS, name = "HEAD PRESSURE")
    for i,j in mstree_dir.edges():
#Hazen Williams equation, A = 10700, C=140, P472
        #m.addConstr(J[i,j] == 
                #10700*(edge_list[i,j][0]/140)**1.852*gp.quicksum(d[i,j,k]/(k**4.8704) for k in pipesize)
               #, "Hydro loss")
        m.addConstr(HL[i,j] == 10700*(edge_list[i,j][0]/140)**1.852*gp.quicksum(d[i,j,k]/(k**4.8704) for k in pipesize)*1000*mstree_dir[i][j]["weight"])
        m.addConstr(HL[i,j] <= HLmax)
#only one pipe per edge
        m.addConstr((d.sum(i, j, '*') == 1.0), "single size chosen") 
    
#only one pump per edge
        m.addConstr((p.sum(i, j, '*') <= 1.0), "single type chosen")
    
#injected pressure pr[i,j] must be less than a huge number.
        #m.addConstr(pr[i,j]<=p[i,j]*10**6)
    
    # injected pressure pr[i,j] must be less than the capacity of pump.
        m.addConstr(pr[i,j] <= p[i,j]*100, "pressure by pump less than 100 m ")
    # energy requirement
        m.addConstr(H[j] == H[i] - HL[i,j] + e[i] - e[j] + pr[i,j], "Energy balance")
    
        #if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) < len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node i is nearer to tank        
            #m.addConstr(H[j] == H[i] -HL[i,j] + e[i] - e[j] + pr[i,j], "Energy balance")
        #if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node j is nearer to tank        
            #m.addConstr(H[i] == H[j] -HL[i,j] + e[j] - e[i] + pr[i,j], "Energy balance")
# Velocity must be between 0.6 and 3 m/s to 30
        m.addConstr((
           edge_list[i,j][0] <= ((3.14/4)*gp.quicksum(d[i,j,k]*k**2 for k in pipesize)) * 3), "Velocity Max Constr"+ str([i,j]))
        #m.addConstr((
           #edge_list[i,j][0] >= ((3.14/4)*gp.quicksum(d[i,j,k]*k**2 for k in pipesize)) * 0.6), "Velocity Min Constr" + str([i,j]))  
    for i in range(len(cluster_elv)):
#pipe elevation must be betwenn lb and ub
        m.addConstr(e[i] >= elevation_lb[i], "PIPE ELV LB")
        m.addConstr(e[i] <= elevation_ub[i], "PIPE ELV UB")
    
        m.addConstr(H[i] >= 31.646232670592248, "Min head pressure") #45 psi, covert it to m
        m.addConstr(H[i] <= 56.26, "Max head pressure") #80 psi, covert it to m   
        
        
    #m.addConstr(H[tank_node] == 31.646232670592248)                
                      
    # pipe cost 
    obj1 = gp.quicksum(1000 * mstree_dir[i][j]["weight"] * gp.quicksum(pipecost[str(k)] * d[i, j, k] 
                                                           for k in pipesize) for i,j in mstree_dir.edges())
    
    # Capital cost of pump stations
    obj2 = gp.quicksum(p.sum(i, j) * (capital_cost_pump_station + ps_flow_cost*edge_list[i,j][0]) for i,j in mstree_dir.edges())
    #Operation and maintenance costs
    obj3 = gp.quicksum(p.sum(i, j)*ps_OM_cost for i, j in mstree_dir.edges())
    # Excavation cost
    obj4 = gp.quicksum((1 + gp.quicksum(d[i,j,k]*k for k in pipesize)) *1000* mstree_dir[i][j]["weight"] * bedding_cost_sq_ft + excavation * \
                               (1 + gp.quicksum(d[i,j,k]*k for k in pipesize)) *1000* mstree_dir[i][j]["weight"] * 0.5 * ((GE[i] - e[i]) + (GE[j] - e[j])) for i, j in mstree_dir.edges())

    # Capital cost of pump stations
    #obj2 = gp.quicksum(1000 * mstree_dir[i][j]["weight"] * gp.quicksum(pumpcost[pumptype.index(m)] * p[i, j, m] 
                                                           #for m in pumptype) for i,j in mstree_dir.edges()) #+penalty

    #Operation and maintenance costs
    #obj3 = gp.quicksum(p.sum(i, j)*ps_OM_cost for i, j in mstree_dir.edges())

    obj = obj1+obj2+obj3+obj4                  
    m.setObjective(obj, GRB.MINIMIZE)
    m.optimize()
    
    
    # diameter
    diameter = { k : v.X for k,v in d.items()}
    diameter = { k : v.X for k,v in d.items() if v.X ==1}
    diameter = list(diameter.keys())
    d = []
    for i in range(len(diameter)):
        d.append(diameter[i][2])
        
    # pump
    pump = { k : v.X for k,v in p.items() if v.X ==1}
    pump = list(pump.keys())
    p = [x[0] for x in pump]
    color_map = []
    for node in mstree[cluster_labels]:
        if node in p:
            color_map.append('blue')
        else: 
            color_map.append('c')
        if node == tank_node:
            color_map[node]=('red') 

            
    #Hydro loss
    hydro_loss = { k : v.X for k,v in HL.items()}

    
    #elev
    ele = { k : v.X for k,v in e.items()}
    
    #head pressure
    h = { k : v.X for k,v in H.items()}
        
    
    return d, color_map, diameter, edge_list, hydro_loss, ele, h
                      

In [8]:
#convert data to dataframe
df = pd.DataFrame(building_coords,
                  columns = ['longitude','latitude','elevation'])
g_mst = createMSTnx(df,1)
dia = []
pum = []
for i in range(0,1):
    output = result(df, 1, i, pipesize, pipecost, 1.095E-5, 1e+5, excavation, bedding_cost_sq_ft, capital_cost_pump_station, ps_OM_cost)
    dia.append(output[0])
    pum.append(output[1])

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-14
Set parameter TimeLimit to value 12000
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12984 rows, 21242 columns and 63724 nonzeros
Model fingerprint: 0x72ecb725
Model has 30680 quadratic objective terms
Variable types: 4722 continuous, 16520 integer (16520 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+05]
  Objective range  [1e+02, 1e+06]
  QObjective range [1e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-05, 1e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 9444 rows and 1873 columns
Presolve time: 0.07s
Presolved: 32834 rows, 48663 columns, 125166 nonzeros
Variable types: 32836 continuous, 15827 integer (15827 binary)
Found heuristic solution: objective 2.794975

 41695 10157 3716762.41  321   24 3716979.06 3715158.34  0.05%   5.3  126s
*41753  8957             379    3716892.9447 3715158.34  0.05%   5.3  126s
 44673 10972 infeasible  124      3716892.94 3715207.55  0.05%   5.2  131s
H46674 11439                    3716869.4369 3715225.54  0.04%   5.1  133s
 46995 12311 3715623.58   88  129 3716869.44 3715226.35  0.04%   5.1  135s
 49928 14986 3716614.71  108  119 3716869.44 3715246.15  0.04%   5.0  140s
*50853 14913             320    3716864.8687 3715248.17  0.04%   5.0  140s
 54596 18369 3716588.86  123  109 3716864.87 3715268.81  0.04%   4.9  146s
 56628 19697 3716813.56  171   72 3716864.87 3715274.40  0.04%   4.8  150s
*57774 19593             310    3716859.4922 3715283.12  0.04%   4.8  150s
H59433 18098                    3716749.7384 3715290.46  0.04%   4.8  154s
H60050 16930                    3716714.6986 3715296.27  0.04%   4.8  178s
 60054 16920 3715976.96   81  167 3716714.70 3715296.27  0.04%   4.8  180s
 60085 16941 3716627.44  

 213691 82020     cutoff  123      3716607.02 3715793.63  0.02%   6.8  617s
 215362 82675 3715984.75  133  141 3716607.02 3715795.11  0.02%   6.8  620s
 218125 83898 3716460.47  142  127 3716607.02 3715796.98  0.02%   6.9  625s
 222360 85610 3716502.28  151  125 3716607.02 3715799.77  0.02%   6.9  632s
 224922 86514     cutoff  116      3716607.02 3715801.82  0.02%   6.9  637s
 227825 87509     cutoff  127      3716607.02 3715804.12  0.02%   6.9  642s
 228921 88280 3716033.86  140  131 3716607.02 3715804.68  0.02%   6.9  645s
 231894 89300     cutoff  140      3716607.02 3715806.44  0.02%   6.9  650s
 234534 90081 3716432.78  131  115 3716607.02 3715808.22  0.02%   6.9  655s
 238351 91846 3716194.03  129  129 3716607.02 3715810.87  0.02%   7.0  662s
 240883 92978 3716261.26  133  121 3716607.02 3715812.55  0.02%   7.0  667s
 242528 93621 3716479.88  128  147 3716607.02 3715813.22  0.02%   7.0  670s
 246667 95152 3715836.11  125  131 3716607.02 3715815.95  0.02%   7.0  676s
 249097 9632

 495898 180196 3716398.04  118  135 3716606.74 3715934.08  0.02%   7.5 1145s
 497898 180883 3716293.15  131  129 3716606.74 3715934.83  0.02%   7.5 1151s
 499809 181585 3716096.43  131  141 3716606.74 3715935.35  0.02%   7.5 1156s
 501264 182150 3716395.30  140  115 3716606.74 3715935.82  0.02%   7.5 1160s
 504398 182939 3716188.03  125  127 3716606.74 3715937.04  0.02%   7.5 1165s
 507409 183663 3716511.92  133  127 3716606.74 3715938.07  0.02%   7.5 1171s
 510504 184506 3716388.47  157  117 3716606.74 3715939.21  0.02%   7.5 1176s
 513260 185264 3716420.98  125  137 3716606.74 3715940.27  0.02%   7.5 1181s
 515313 185895 3716525.18  135  110 3716606.74 3715941.04  0.02%   7.5 1185s
 517968 186883     cutoff  140      3716606.74 3715942.08  0.02%   7.5 1191s
 520524 187655 3716368.09  120  143 3716606.74 3715943.00  0.02%   7.5 1196s
 523162 188421 3716584.17  153  119 3716606.74 3715944.01  0.02%   7.6 1201s
 525671 188980     cutoff  129      3716606.74 3715945.04  0.02%   7.6 1206s

 760343 250287 3716579.19  148  108 3716606.74 3716021.22  0.02%   7.8 1692s
 761860 250545     cutoff  120      3716606.74 3716021.58  0.02%   7.8 1695s
 764471 251354 3716459.96  124  134 3716606.74 3716022.28  0.02%   7.8 1701s
 765568 251795     cutoff  129      3716606.74 3716022.82  0.02%   7.8 1705s
 768107 252530     cutoff  130      3716606.74 3716023.52  0.02%   7.8 1711s
 771181 253321 3716442.61  131  135 3716606.74 3716024.37  0.02%   7.8 1716s
 774261 253742 3716364.06  128  137 3716606.74 3716025.16  0.02%   7.8 1722s
 775344 254051     cutoff  132      3716606.74 3716025.40  0.02%   7.8 1726s
 777519 254739     cutoff  124      3716606.74 3716026.05  0.02%   7.8 1732s
 779866 255036 3716061.45  133  123 3716606.74 3716026.51  0.02%   7.8 1735s
 781751 255538     cutoff  134      3716606.74 3716027.23  0.02%   7.8 1740s
 784607 256379 3716245.53  133  139 3716606.74 3716027.92  0.02%   7.8 1746s
 787263 257050 3716501.01  122  135 3716606.74 3716028.60  0.02%   7.8 1752s

 1001575 308922     cutoff  126      3716606.74 3716079.64  0.01%   8.0 2227s
 1004271 309589 3716503.65  136  139 3716606.74 3716080.30  0.01%   8.0 2232s
 1005912 309844 3716250.15  118  151 3716606.74 3716080.61  0.01%   8.0 2235s
 1008603 310424 3716174.48  136  135 3716606.74 3716081.16  0.01%   8.0 2240s
 1011385 311013 3716104.06  131  145 3716606.74 3716081.45  0.01%   8.0 2245s
 1012882 311511 3716294.83  108  151 3716606.74 3716082.14  0.01%   8.0 2250s
 1015502 312227     cutoff  138      3716606.74 3716082.56  0.01%   8.0 2256s
 1018375 312905     cutoff  138      3716606.74 3716083.20  0.01%   8.0 2262s
 1019535 313229 3716557.36  150  135 3716606.74 3716083.52  0.01%   8.0 2265s
 1022584 313805 3716493.50  143  150 3716606.74 3716084.07  0.01%   8.0 2270s
 1024592 314371 3716477.39  126  147 3716606.74 3716084.52  0.01%   8.0 2276s
 1027162 314863     cutoff  128      3716606.74 3716085.08  0.01%   8.0 2281s
 1029733 315587 3716481.40  122  143 3716606.74 3716085.51  0.01

 1243435 361585 3716378.59  141  155 3716606.74 3716125.29  0.01%   8.2 2961s
 1246189 362150 3716407.16  140  132 3716606.74 3716125.67  0.01%   8.2 2966s
 1248882 362721     cutoff  130      3716606.74 3716126.10  0.01%   8.2 2971s
 1251264 363461 3716543.28  129  143 3716606.74 3716126.57  0.01%   8.2 2977s
 1253834 363937 3716597.35  121  140 3716606.74 3716126.89  0.01%   8.2 2982s
 1256447 364421 3716250.55  128  153 3716606.74 3716127.35  0.01%   8.2 2987s
 1258657 364996     cutoff  119      3716606.74 3716127.69  0.01%   8.2 2992s
 1260168 365258 3716589.54  160  105 3716606.74 3716127.85  0.01%   8.2 2995s
 1262746 365687 3716500.36  117  152 3716606.74 3716128.34  0.01%   8.2 3001s
 1265279 366154 3716180.74  115  146 3716606.74 3716128.82  0.01%   8.2 3005s
 1267742 366736     cutoff  137      3716606.74 3716129.17  0.01%   8.2 3010s
 1270257 367212 3716382.98  133  143 3716606.74 3716129.55  0.01%   8.2 3016s
 1272853 367923 3716557.26  128  127 3716606.74 3716129.97  0.01

 1478474 411563 3716541.06  151  135 3716606.74 3716160.90  0.01%   8.3 3485s
 1481307 411961     cutoff  130      3716606.74 3716161.34  0.01%   8.3 3490s
 1483713 412613 3716578.40  149  108 3716606.74 3716161.66  0.01%   8.3 3495s
 1486498 413152 3716265.24  126  151 3716606.74 3716162.12  0.01%   8.3 3500s
 1489181 413749 3716349.65  122  147 3716606.74 3716162.45  0.01%   8.3 3505s
 1491805 414277     cutoff  132      3716606.74 3716162.84  0.01%   8.3 3511s
 1494870 414579 3716570.21  129  146 3716606.74 3716163.29  0.01%   8.3 3515s
 1496888 414991 3716389.80  122  137 3716606.74 3716163.58  0.01%   8.3 3520s
 1499517 415510     cutoff  123      3716606.74 3716163.97  0.01%   8.3 3525s
 1501773 416150 3716416.41  130  159 3716606.74 3716164.21  0.01%   8.3 3530s
 1504856 416789 3716582.61  157  118 3716606.74 3716164.55  0.01%   8.3 3536s
 1507544 417284 3716567.82  129  135 3716606.74 3716164.88  0.01%   8.3 3541s
 1510045 417785 3716585.14  141  140 3716606.74 3716165.23  0.01

 1749048 473375 3716247.09  139  133 3716606.74 3716193.30  0.01%   8.4 4026s
 1751675 473905 3716492.04  156  115 3716606.74 3716193.60  0.01%   8.4 4031s
 1754040 474680 3716593.59  153  117 3716606.74 3716193.92  0.01%   8.4 4037s
 1755701 474894 infeasible  126      3716606.74 3716194.07  0.01%   8.4 4040s
 1758435 475473 3716425.18  122  141 3716606.74 3716194.40  0.01%   8.4 4046s
 1760331 475983     cutoff  120      3716606.74 3716194.64  0.01%   8.4 4051s
 1763139 476594     cutoff  116      3716606.74 3716194.92  0.01%   8.4 4056s
 1765869 477371 3716497.51  129  141 3716606.74 3716195.30  0.01%   8.4 4061s
 1768542 478042 3716285.90  141  137 3716606.74 3716195.60  0.01%   8.4 4067s
 1771259 478475 3716582.74  119  145 3716606.74 3716195.92  0.01%   8.4 4072s
 1774056 479232 3716422.69  131  157 3716606.74 3716196.21  0.01%   8.4 4077s
 1776495 479797 3716482.98  114  134 3716606.74 3716196.45  0.01%   8.4 4082s
 1779085 480465 3716526.90  154  123 3716606.74 3716196.71  0.01

 2001165 534793 3716296.30  135  135 3716606.74 3716219.37  0.01%   8.3 4970s
 2003743 535509     cutoff  143      3716606.74 3716219.60  0.01%   8.3 4975s
 2006305 536053     cutoff  126      3716606.74 3716219.89  0.01%   8.3 4980s
 2008777 536602 3716452.29  118  155 3716606.74 3716220.14  0.01%   8.3 4985s
 2011285 537322 3716443.43  142  135 3716606.74 3716220.38  0.01%   8.3 4990s
 2014201 537907 3716506.28  130  151 3716606.74 3716220.70  0.01%   8.3 4995s
 2016846 538566 3716386.65  151  107 3716606.74 3716220.91  0.01%   8.3 5001s
 2019443 539076     cutoff  134      3716606.74 3716221.21  0.01%   8.3 5005s
 2021687 539693 3716572.77  155  127 3716606.74 3716221.45  0.01%   8.3 5011s
 2024355 540198 3716499.17  142  129 3716606.74 3716221.69  0.01%   8.3 5016s
 2026942 541094 3716341.42  140  149 3716606.74 3716221.92  0.01%   8.3 5021s
 2029845 541591 3716426.00  126  150 3716606.74 3716222.21  0.01%   8.3 5026s
 2032855 542234 3716289.33  118  142 3716606.74 3716222.48  0.01

In [15]:
def p_joint(n):
    if n == 0:
        return pum[0]
    else:
        p_all = pum[n]+p_joint(n-1)
        return p_all

In [16]:
def d_joint(n):
    if n == 0:
        return dia[0]
    else:
        d_all = dia[n]+d_joint(n-1)
        return d_all

In [17]:
def g_joint(n):
    if n == 0:
        return g_mst[0][0]
    else:
        g = nx.disjoint_union(g_joint(n-1),g_mst[0][n])
        return g

In [21]:
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

print(find_indices(p_joint(0), 'red'))

[901]


In [19]:
pos = {city:(long, lat) for (city, (lat,long)) in nx.get_node_attributes(g_joint(0), 'pos').items()}
plt.figure(3,figsize=(120,120)) 
nx.draw(g_joint(0), pos, with_labels=True, 
        node_size=600, node_color = p_joint(0), 
        edgelist=g_joint(0).edges(),edge_color = d_joint(0),
        edge_cmap=plt.cm.Blues, width=10)
# node legend
pump_patch = mlines.Line2D([], [], color='blue', marker='o', linestyle='None',
                          markersize=100, label='Pump')
tank_patch = mlines.Line2D([], [], color='red', marker='o', linestyle='None',
                          markersize=100, label='Tank')
nonpump_patch = mlines.Line2D([], [], color='c', marker='o', linestyle='None',
                          markersize=100, label='Non-pump')
plt.legend(handles=[pump_patch, tank_patch, nonpump_patch],fontsize=100)
#plt.title('The least cost system sizing for 1 subnework',fontsize=200)
plt.show()